# `Dataset`

> A python list like object that contains your evaluation data.

In [ ]:
# | default_exp dataset

In [1]:
# | hide

from unittest.mock import MagicMock
from fastcore.test import *

In [3]:
# | export
import typing as t

from fastcore.utils import patch

from pydantic import BaseModel

from ragas_annotator.model.notion_model import NotionModel
from ragas_annotator.backends.ragas_api_client import RagasApiClient
from ragas_annotator.backends.notion_backend import NotionBackend

In [ ]:
# | export
PydanticModelType = t.TypeVar("PydanticModelType", bound=BaseModel)


class Dataset(t.Generic[PydanticModelType]):
    """A list-like interface for managing NotionModel instances in a Notion database."""

    def __init__(
        self,
        name: str,
        model: t.Type[BaseModel],
        dataset_id: str,
        ragas_api_client: RagasApiClient,
    ):
        self.name = name
        self.model = model
        self.dataset_id = dataset_id
        self._ragas_api_client = ragas_api_client
        self._entries: t.List[PydanticModelType] = []

    def __getitem__(
        self, key: t.Union[int, slice]
    ) -> t.Union[PydanticModelType, "Dataset[PydanticModelType]"]:
        """Get an entry by index or slice."""
        if isinstance(key, slice):
            new_dataset = type(self)(
                name=self.name,
                model=self.model,
                dataset_id=self.dataset_id,
                ragas_api_client=self._ragas_api_client,
            )
            new_dataset._entries = self._entries[key]
            return new_dataset
        else:
            return self._entries[key]

    def __setitem__(self, index: int, entry: PydanticModelType) -> None:
        """Update an entry at the given index and sync to Notion."""
        if not isinstance(entry, self.model):
            raise TypeError(f"Entry must be an instance of {self.model.__name__}")

        # Get existing entry to get Notion page ID
        existing = self._entries[index]
        if not hasattr(existing, "_page_id"):
            raise ValueError("Existing entry has no page_id")

        # Update in Notion
        assert (
            existing._page_id is not None
        )  # mypy fails to infer that we check for it above
        response = self._ragas_api_client.update_page(
            page_id=existing._page_id, properties=entry.to_notion()["properties"]
        )

        # Update local cache with response data
        self._entries[index] = self.model.from_notion(response)

    def __repr__(self) -> str:
        return (
            f"Dataset(name={self.name}, model={self.model.__name__}, len={len(self)})"
        )

    def __len__(self) -> int:
        return len(self._entries)

    def __iter__(self) -> t.Iterator[PydanticModelType]:
        return iter(self._entries)

In [ ]:
# | hide
import ragas_annotator.model.notion_typing as nmt
from ragas_annotator.backends.mock_notion import MockNotionClient
from ragas_annotator.backends.factory import NotionClientFactory
from ragas_annotator.backends.notion_backend import NotionBackend

In [ ]:
# test model
class TestModel(NotionModel):
    id: int = nmt.ID()
    name: str = nmt.Title()
    description: str = nmt.Text()


test_model = TestModel(name="test", description="test description")
test_model

TestModel(name='test' description='test description')

In [ ]:
# | hide
# Set up a test environment with mock Notion client and a test database.
# root page id
root_page_id = "test-root-id"
# Create a mock client
mock_client = NotionClientFactory.create(
    use_mock=True, initialize_project=True, root_page_id=root_page_id
)

# Create NotionBackend with mock client
backend = NotionBackend(root_page_id=root_page_id, notion_client=mock_client)

# get the page id of the datasets page
dataset_page_id = backend.get_page_id(parent_id=root_page_id, page_name="Datasets")

# create a new database in the datasets page
properties = {}
for _, field in TestModel._fields.items():
    properties.update(field._to_notion_property())
datasets_id = backend.create_new_database(
    parent_page_id=dataset_page_id, title="TestModel", properties=properties
)

In [ ]:
dataset = Dataset(
    name="TestModel", model=TestModel, dataset_id=datasets_id, ragas_api_client=backend
)

In [ ]:
# | export
@patch
def append(self: Dataset, entry: PydanticModelType) -> None:
    """Add a new entry to the dataset and sync to Notion."""
    # if not isinstance(entry, self.model):
    #     raise TypeError(f"Entry must be an instance of {self.model.__name__}")

    # Create in Notion and get response
    response = self._ragas_api_client.create_page_in_database(
        database_id=self.dataset_id, properties=entry.to_notion()["properties"]
    )

    # Update entry with Notion data (like ID)
    updated_entry = self.model.from_notion(response)
    self._entries.append(updated_entry)

In [ ]:
dataset.append(test_model)
len(dataset)

1

In [ ]:
# | hide
test_eq(len(dataset), 1)

In [ ]:
# | export
@patch
def pop(self: Dataset, index: int = -1) -> PydanticModelType:
    """Remove and return entry at index, sync deletion to Notion."""
    entry = self._entries[index]
    if not hasattr(entry, "_page_id"):
        raise ValueError("Entry has no page_id")

    # Archive in Notion (soft delete)
    assert entry._page_id is not None  # mypy fails to infer that we check for it above
    self._ragas_api_client.update_page(page_id=entry._page_id, archived=True)

    # Remove from local cache
    return self._entries.pop(index)

In [ ]:
dataset.pop()
len(dataset)

0

In [ ]:
# | hide
test_eq(len(dataset), 0)

In [ ]:
# | export
@patch
def load(self: Dataset) -> None:
    """Load all entries from the Notion database."""
    # Query the database
    response = self._ragas_api_client.query_database(
        database_id=self.dataset_id, archived=False
    )

    # Clear existing entries
    self._entries.clear()

    # Convert results to model instances
    for page in response.get("results", []):
        entry = self.model.from_notion(page)
        self._entries.append(entry)

In [ ]:
dataset.load()

In [ ]:
for i in range(3):
    dataset.append(test_model)
len(dataset)

3

In [ ]:
# create a new instance of the dataset
dataset = Dataset(
    name="TestModel",
    model=TestModel,
    dataset_id=datasets_id,
    ragas_api_client=backend,
)
len(dataset)

0

In [ ]:
dataset.load()
test_eq(len(dataset), 3)

In [ ]:
# | export
@patch
def get(self: Dataset, id: int) -> t.Optional[PydanticModelType]:
    """Get an entry by ID."""
    if not self._ragas_api_client:
        return None

    # Query the database for the specific ID
    response = self._ragas_api_client.query_database(
        database_id=self.dataset_id,
        filter={"property": "id", "unique_id": {"equals": id}},
    )

    if not response.get("results"):
        return None

    return self.model.from_notion(response["results"][0])

In [ ]:
test_model = dataset.get(0)
test_model

TestModel(name='test' description='test description')

In [ ]:
# | hide
test_eq(test_model.description, "test description")

In [ ]:
# | export
@patch
def save(self: Dataset, item: PydanticModelType) -> None:
    """Save changes to an item to Notion."""
    if not isinstance(item, self.model):
        raise TypeError(f"Item must be an instance of {self.model.__name__}")

    if not hasattr(item, "_page_id"):
        raise ValueError("Item has no page_id")

    # Update in Notion
    assert item._page_id is not None  # mypy fails to infer that we check for it above
    response = self._ragas_api_client.update_page(
        page_id=item._page_id, properties=item.to_notion()["properties"]
    )

    # Update local cache
    for i, existing in enumerate(self._entries):
        if existing._page_id == item._page_id:
            self._entries[i] = self.model.from_notion(response)
            break

In [ ]:
test_model.description = "updated description"
dataset.save(test_model)

In [ ]:
dataset.get(0)

TestModel(name='test' description='updated description')

In [ ]:
# | hide
test_eq(dataset.get(0).description, "updated description")